## Import dependencies

In [2]:
import os

import pandas as pd
import numpy as np

from flask import Flask, render_template, jsonify, request
from flask_sqlalchemy import SQLAlchemy

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

import sqlite3

## Database setup

In [3]:
conn = sqlite3.connect("./Resources/gdp_olympic.sqlite")
cursor = conn.cursor()

## Data retrieval and cleanup

### Join two main tables

In [4]:
query = '''
    SELECT wdi.year, wdi.country_name, wdi.country_code, winter.medal, wdi.pop_total
    FROM winter INNER JOIN wdi 
    ON winter.country_code = wdi.country_code
    WHERE winter.year = wdi.year
'''

df = pd.read_sql_query(query, conn)

### Values: x-axis

In [5]:
# Create list of x-values
years = df.year.unique().tolist()

In [6]:
# Print list
years

[1960,
 1964,
 1968,
 1972,
 1976,
 1980,
 1984,
 1988,
 1992,
 1994,
 1998,
 2002,
 2006,
 2010,
 2014]

### Values: y-axis (population percentages)

In [7]:
# Create list where each element corresponds to the total population of countries who medaled in that year's 
# winter olympics, starting from 1960 up until 2014

pop_totals = [np.sum(df[df.year == year].pop_total.unique()) for year in years]

In [8]:
# Print list
pop_totals

[488904116,
 548730325,
 598666607,
 772626943,
 670098141,
 823096375,
 807342623,
 784256801,
 2079020543,
 2220271838,
 2297082213,
 2317875405,
 2417280969,
 2427557996,
 2517198631]

In [9]:
# Add empty column to dataframe to fill with population percentages
df['pop_percentage'] = ''

In [10]:
# Print head
df.head()

,year,country_name,country_code,medal,pop_total,pop_percentage
0,1960,Russian Federation,RUS,Bronze,119897000,
1,1960,Sweden,SWE,Gold,7484656,
2,1960,Finland,FIN,Silver,4429634,
3,1960,Russian Federation,RUS,Bronze,119897000,
4,1960,Russian Federation,RUS,Bronze,119897000,


In [11]:
# Populate this column
for i in range(len(df)):
    year = df.iloc[i, 0]
    index = years.index(year)
    df.iloc[i, 5] = np.round(df.iloc[i, 4] / pop_totals[index], 4)

In [12]:
# Print head
df.head()

,year,country_name,country_code,medal,pop_total,pop_percentage
0,1960,Russian Federation,RUS,Bronze,119897000,0.2452
1,1960,Sweden,SWE,Gold,7484656,0.0153
2,1960,Finland,FIN,Silver,4429634,0.0091
3,1960,Russian Federation,RUS,Bronze,119897000,0.2452
4,1960,Russian Federation,RUS,Bronze,119897000,0.2452


In [13]:
# Group df by year and country and reset index in the process
population = pd.DataFrame(df.groupby(['year', 'country_code']).max()['pop_percentage']).reset_index()

In [14]:
# Print head
population.head()

,year,country_code,pop_percentage
0,1960,AUT,0.0144
1,1960,CAN,0.0366
2,1960,CZE,0.0196
3,1960,FIN,0.0091
4,1960,FRA,0.0954


### Values: y-axis (medal percentages)

In [15]:
# Build a dataframe which counts number of medals by year and country and reset index in the process
medals = pd.DataFrame(df.groupby(['year', 'country_code']).count()['medal']).reset_index()

In [16]:
# Add empty column 
medals['medal_percentage'] = ''

In [17]:
# Print head
medals.head()

,year,country_code,medal,medal_percentage
0,1960,AUT,6,
1,1960,CAN,21,
2,1960,CZE,1,
3,1960,FIN,13,
4,1960,FRA,3,


In [18]:
# Create series indexed by year with values the total number of medals given out that winter games
medal_series = df.groupby('year').count()['medal'] 

In [19]:
# Populate empty column
for i in range(len(medals)):
    year = medals.iloc[i, 0]
    medals.iloc[i, 3] = np.round(medals.iloc[i, 2] / medal_series[year], 4)

In [20]:
# Print head
medals.head(10)

,year,country_code,medal,medal_percentage
0,1960,AUT,6,0.0408
1,1960,CAN,21,0.1429
2,1960,CZE,1,0.0068
3,1960,FIN,13,0.0884
4,1960,FRA,3,0.0204
5,1960,ITA,1,0.0068
6,1960,NED,2,0.0136
7,1960,NOR,9,0.0612
8,1960,POL,2,0.0136
9,1960,RUS,42,0.2857


### Form dictionaries for y-axis data

In [21]:
# Join population and medals dataframes
y_axis_data = population.merge(medals, how='inner', on='country_code')

In [22]:
# Print head
y_axis_data.head()

,year_x,country_code,pop_percentage,year_y,medal,medal_percentage
0,1960,AUT,0.0144,1960,6,0.0408
1,1960,AUT,0.0144,1964,17,0.0919
2,1960,AUT,0.0144,1968,15,0.0754
3,1960,AUT,0.0144,1972,5,0.025
4,1960,AUT,0.0144,1976,7,0.0333


In [23]:
# Filter so that there is only one row per country and year pairing
y_axis_data = y_axis_data[y_axis_data.year_x == y_axis_data.year_y]

In [24]:
# Drop columns
y_axis_data.drop(axis=1, columns=['year_y', 'medal'], inplace=True)

In [25]:
# Redefine first column header to just be 'year'
y_axis_data.rename(columns={'year_x': 'year'}, inplace=True)

In [26]:
# Print head
y_axis_data.head()

,year,country_code,pop_percentage,medal_percentage
0,1960,AUT,0.0144,0.0408
16,1964,AUT,0.0132,0.0919
32,1968,AUT,0.0124,0.0754
48,1972,AUT,0.0098,0.025
64,1976,AUT,0.0113,0.0333


In [27]:
# Convert all datatypes to strings to export to csv (and because JSON cannot take in numpy datatypes as input)
y_axis_data = y_axis_data.astype(str) 

In [28]:
# Export csv for app.py to call, instead of utilizing all the code above at run time
y_axis_data.to_csv('./Resources/line_graph.csv')